### 1. csv 파일 불러오기

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from deepchem.feat.molecule_featurizers import RDKitDescriptors
from rdkit import Chem
from rdkit.Chem import Descriptors
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [3]:
warnings.filterwarnings('ignore')           # 출력 창 경고 메시지 무시
plt.rcParams['axes.unicode_minus'] = False  # 음수부호 깨짐 관련
plt.rcParams['font.family'] = 'Malgun Gothic' # window 전용 한글 폰트 설정

In [4]:
train_df = pd.read_csv('../rdkit_data/rdkit_train.csv')
test_df = pd.read_csv('../rdkit_data/rdkit_test.csv')

### 2. 데이터 전처리

In [5]:
zero_ratio = (train_df == 0).mean()
columns_to_drop = zero_ratio[zero_ratio >= 2/3].index
df = train_df.drop(columns=columns_to_drop)

In [6]:
min_value = 268.28
max_value = 602.65

# 범위 바깥의 행 제거
df_filtered = df[(df['MolWt'] >= min_value) & (df['MolWt'] <= max_value)]

In [7]:
df_filtered

,Smiles,pIC50,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,...,fr_C_O_noCOO,fr_NH0,fr_NH1,fr_amide,fr_aniline,fr_benzene,fr_bicyclic,fr_ether,fr_halogen,fr_pyridine
1,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,10.59,14.673248,14.673248,0.205025,-1.585128,0.479279,19.333333,535.624,501.352,...,2,6,1,2,2,1,2,0,1,0
2,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,10.11,14.663674,14.663674,0.195978,-1.596212,0.466884,19.333333,537.596,505.340,...,2,6,1,2,2,1,2,1,1,0
3,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,10.09,14.610375,14.610375,0.119300,-2.432700,0.448013,17.256410,545.566,515.326,...,2,6,1,2,2,1,2,0,3,0
5,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,10.00,14.673674,14.673674,0.133545,-2.594941,0.449340,20.292683,571.604,539.348,...,2,6,1,2,2,1,2,0,3,0
6,COc1cc2nn([C@H]3CC[C@@]4(CCC(=O)N4C)CC3)cc2cc1...,10.00,13.169072,13.169072,0.015212,-0.307871,0.486630,22.400000,473.537,446.321,...,2,6,1,2,1,1,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,O=C1c2oc(-c3ccc(Cl)cc3)cc2Cc2cc(N3CCNCC3)ccc2N...,4.58,14.317048,14.317048,0.143634,-0.143634,0.255392,14.902439,561.085,531.853,...,1,3,1,1,2,3,2,0,1,1
1947,O=C(Nc1nc2cc[nH]cc-2n1)c1cccc([N+](=O)[O-])c1,4.52,12.044186,12.044186,0.147374,-0.555627,0.563801,10.476190,283.247,274.175,...,1,3,2,1,1,1,1,0,0,1
1948,CCCCn1c(NC(=O)c2cccc(Cl)c2)nc2ccccc21,4.52,12.446045,12.446045,0.207820,-0.207820,0.735215,10.869565,327.815,309.671,...,1,2,1,1,1,2,1,0,1,0
1949,O=C(Nc1nc2cc(F)c(F)cc2[nH]1)c1cccc([N+](=O)[O-...,4.52,13.131519,13.131519,0.026383,-1.055036,0.572289,10.695652,318.239,310.175,...,1,2,2,1,1,2,1,0,2,0


In [8]:
df_filtered_dr = df_filtered.loc[:,
    ~df_filtered.columns.str.startswith('fr_') & 
    ~df_filtered.columns.str.startswith('Num') & 
    ~df_filtered.columns.str.endswith('Count')]

In [9]:
df_filtered_dr.drop(['Smiles'], axis=1, inplace = True)

In [10]:
df_filtered_dr

,pIC50,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,...,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,FractionCSP3,MolLogP,MolMR
1,10.59,14.673248,14.673248,0.205025,-1.585128,0.479279,19.333333,535.624,501.352,535.270716,...,37.181863,17.355847,1.858160,-0.605817,5.973562,6.530311,6.735552,0.500000,2.7209,144.7415
2,10.11,14.663674,14.663674,0.195978,-1.596212,0.466884,19.333333,537.596,505.340,537.249981,...,36.946453,17.313219,1.709236,-0.643248,5.788572,3.208629,7.377535,0.481481,1.5672,141.7095
3,10.09,14.610375,14.610375,0.119300,-2.432700,0.448013,17.256410,545.566,515.326,545.236222,...,35.678557,17.097310,1.004424,-0.826426,5.056355,0.555242,3.820176,0.461538,2.4335,137.9735
5,10.00,14.673674,14.673674,0.133545,-2.594941,0.449340,20.292683,571.604,539.348,571.251873,...,36.384732,17.206793,1.184201,-3.376925,5.003609,2.772099,4.753124,0.500000,2.9661,145.0715
6,10.00,13.169072,13.169072,0.015212,-0.307871,0.486630,22.400000,473.537,446.321,473.217538,...,31.490607,12.816851,1.859062,0.883378,7.502523,10.745083,0.000000,0.400000,3.4460,129.4652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,4.58,14.317048,14.317048,0.143634,-0.143634,0.255392,14.902439,561.085,531.853,560.197904,...,22.911638,4.088939,8.086775,0.903744,28.192789,4.237466,4.237891,0.176471,6.8228,163.6712
1947,4.52,12.044186,12.044186,0.147374,-0.555627,0.563801,10.476190,283.247,274.175,283.070539,...,33.315579,13.213323,1.282767,-0.348063,7.180719,3.355674,0.000000,0.000000,2.0700,74.0543
1948,4.52,12.446045,12.446045,0.207820,-0.207820,0.735215,10.869565,327.815,309.671,327.113840,...,16.997567,3.445669,2.429573,0.365908,14.790931,2.105731,2.963804,0.222222,4.7421,94.0462
1949,4.52,13.131519,13.131519,0.026383,-1.055036,0.572289,10.695652,318.239,310.175,318.056447,...,28.663548,13.079237,0.166574,-2.769928,6.936975,0.000000,0.000000,0.000000,3.0016,77.0593


In [11]:
correlation_matrix = df_filtered_dr.corr()
abs_correlation_matrix = correlation_matrix.abs()
filtered_correlation_matrix = abs_correlation_matrix.where(abs_correlation_matrix >= 0.3)
pic50_correlation = filtered_correlation_matrix['pIC50']
pic50_correlation = pic50_correlation.dropna()

In [12]:
pic50_correlation

pIC50             1.000000
MolWt             0.419513
HeavyAtomMolWt    0.412494
ExactMolWt        0.419980
BCUT2D_LOGPLOW    0.426004
BertzCT           0.387467
Chi0              0.490421
Chi0n             0.472446
Chi0v             0.427981
Chi1              0.412116
Chi1n             0.388970
Chi1v             0.306981
Chi2n             0.458627
Chi2v             0.350919
HallKierAlpha     0.370152
Kappa1            0.492431
Kappa2            0.379200
Kappa3            0.431100
LabuteASA         0.419531
PEOE_VSA3         0.370370
PEOE_VSA4         0.334272
PEOE_VSA6         0.353034
PEOE_VSA8         0.322184
PEOE_VSA9         0.300762
SMR_VSA3          0.433778
SMR_VSA5          0.427869
SlogP_VSA2        0.315352
SlogP_VSA5        0.427292
TPSA              0.375597
EState_VSA1       0.308087
VSA_EState1       0.333459
MolMR             0.352372
Name: pIC50, dtype: float64

In [20]:
filtered_correlation_matrix_4 = abs_correlation_matrix.where(abs_correlation_matrix >= 0.4)
pic50_correlation_4 = filtered_correlation_matrix_4['pIC50']
pic50_correlation_4 = pic50_correlation_4.dropna()
pic50_correlation_4

pIC50             1.000000
MolWt             0.419513
HeavyAtomMolWt    0.412494
ExactMolWt        0.419980
BCUT2D_LOGPLOW    0.426004
Chi0              0.490421
Chi0n             0.472446
Chi0v             0.427981
Chi1              0.412116
Chi2n             0.458627
Kappa1            0.492431
Kappa3            0.431100
LabuteASA         0.419531
SMR_VSA3          0.433778
SMR_VSA5          0.427869
SlogP_VSA5        0.427292
Name: pIC50, dtype: float64

### 3. csv 파일 저장

In [21]:
column_names_4 = pic50_correlation_4.index.tolist()

In [22]:
test_column_names_4 = [name for name in pic50_correlation_4.index.tolist() if name != 'pIC50']

In [23]:
train_final_4 = df_filtered_dr[column_names_4]

In [24]:
test_final_4 = test_df[test_column_names_4]

In [25]:
train_final_4.to_csv('../rdkit_data/rdkit_train_non_category_corr_0.4.csv', index=False)
test_final_4.to_csv('../rdkit_data/rdkit_test_non_categorycorr_0.4.csv', index=False)